In [1]:
import numpy as np
import gurobipy as gp

# Simple model without battery

In [3]:
#gp.setParam('OutputFlag', 0)

In [4]:
#%%timeit
model = gp.Model("min_costs")

wind_costs = 1
solar_costs = 1
electrolizer_costs = 1
battery_cost = 1

electrolizer_efficiency = 1

wind_size = model.addVar(name="wind_size")
solar_size = model.addVar(name="solar_size")
electrolyzer_size = model.addVar(name="electrolyzer_size")
battery_size = model.addVar(name="electrolyzer_size")

curtailment = model.addMVar(8760)

model.update()  # workaround: https://stackoverflow.com/a/66212311/859591

costs = (wind_costs * wind_size +
         solar_size * solar_costs +
         electrolyzer_size * electrolizer_costs +
         battery_size * battery_cost)

wind_profile = np.random.random(8760)
solar_profile = np.random.random(8760)

energy_production = wind_profile * wind_size + solar_profile * solar_size
energy_production_yearly = gp.quicksum(energy_production)

for energy_production_h in energy_production:
    model.addConstr(energy_production_h <= electrolyzer_size)
    
model.addConstr(1e5 <= energy_production_yearly, "energy_production_minimum")
model.setObjective(costs, gp.GRB.MINIMIZE)

model.optimize()

Using license file /opt/gurobi810/gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (linux64)
Optimize a model with 8761 rows, 8764 columns and 26282 nonzeros
Model fingerprint: 0x22001fc8
Coefficient statistics:
  Matrix range     [6e-06, 4e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+05, 1e+05]
Presolve removed 0 rows and 8761 columns
Presolve time: 0.01s
Presolved: 8761 rows, 3 columns, 26282 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.562500e+03   0.000000e+00      0s
       5    4.5418425e+01   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.03 seconds
Optimal objective  4.541842510e+01


In [5]:
solar_size

<gurobi.Var solar_size (value 0.45540716312455654)>

In [6]:
wind_size

<gurobi.Var wind_size (value 22.266196012443952)>

# With battery

In [39]:
gp.setParam('OutputFlag', 0)

In [40]:
%%timeit
model = gp.Model("min_costs")

wind_costs = 1
solar_costs = 1
electrolizer_costs = 1
battery_cost = 0.0001

electrolizer_efficiency = 1

wind_size = model.addVar(name="wind_size")
solar_size = model.addVar(name="solar_size")
electrolyzer_size = model.addVar(name="electrolyzer_size")
battery_size = model.addVar(name="electrolyzer_size")

battery_level = model.addVars(8760)
electrolyzer_in = model.addVars(8760)

model.update()  # workaround: https://stackoverflow.com/a/66212311/859591

costs = (wind_costs * wind_size +
         solar_size * solar_costs +
         electrolyzer_size * electrolizer_costs +
         battery_size * battery_cost)

wind_profile = np.zeros(8760)  #np.random.random(8760)
wind_profile[::2] = 1
solar_profile = np.zeros(8760)  #np.random.random(8760)

power_generation_profile = wind_profile * wind_size + solar_profile * solar_size

for t in range(8760):
    model.addConstr(battery_level[t] <= battery_size)
    model.addConstr(electrolyzer_in[t] <= electrolyzer_size)
    model.addConstr(electrolyzer_in[t] <= power_generation_profile[t] + battery_level[t])


model.addConstr(battery_level[0] == 0)
for t in range(1, 8760):
    model.addConstr(battery_level[t] <= power_generation_profile[t-1] + battery_level[t-1] - electrolyzer_in[t-1])


hyrdogen_yearly = gp.quicksum(electrolyzer_in) * electrolizer_efficiency

    
model.addConstr(1e5 <= hyrdogen_yearly, "energy_production_minimum")
model.setObjective(costs, gp.GRB.MINIMIZE)

model.optimize()

2.23 s ± 124 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
battery_level

{0: <gurobi.Var C4 (value 0.0)>,
 1: <gurobi.Var C5 (value 11.415525114155251)>,
 2: <gurobi.Var C6 (value 0.0)>,
 3: <gurobi.Var C7 (value 11.415525114155251)>,
 4: <gurobi.Var C8 (value 0.0)>,
 5: <gurobi.Var C9 (value 11.415525114155251)>,
 6: <gurobi.Var C10 (value 0.0)>,
 7: <gurobi.Var C11 (value 11.415525114155251)>,
 8: <gurobi.Var C12 (value 0.0)>,
 9: <gurobi.Var C13 (value 11.415525114155251)>,
 10: <gurobi.Var C14 (value 0.0)>,
 11: <gurobi.Var C15 (value 11.415525114155251)>,
 12: <gurobi.Var C16 (value 0.0)>,
 13: <gurobi.Var C17 (value 11.415525114155251)>,
 14: <gurobi.Var C18 (value 0.0)>,
 15: <gurobi.Var C19 (value 11.415525114155251)>,
 16: <gurobi.Var C20 (value 0.0)>,
 17: <gurobi.Var C21 (value 11.415525114155251)>,
 18: <gurobi.Var C22 (value 0.0)>,
 19: <gurobi.Var C23 (value 11.415525114155251)>,
 20: <gurobi.Var C24 (value 0.0)>,
 21: <gurobi.Var C25 (value 11.415525114155251)>,
 22: <gurobi.Var C26 (value 0.0)>,
 23: <gurobi.Var C27 (value 11.415525114155251

In [21]:
wind_size

<gurobi.Var wind_size (value 22.831050228310502)>

In [22]:
solar_size

<gurobi.Var solar_size (value 0.0)>

In [23]:
battery_size

<gurobi.Var electrolyzer_size (value 11.415525114155251)>

In [31]:
model.x

[22.831050228310502,
 0.0,
 11.415525114155251,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415

In [37]:
duration_one_run = 100e-3
samples = 1000
#samples = 100_000

to_days = 1 / 3600 / 24

cores = 20 * 32                                                            

duration_one_run * samples * to_days / cores * 5**6 

0.028257016782407406

# With battery efficiency and capped dis/charging rate

In [39]:
gp.setParam('OutputFlag', 0)

In [49]:
%%timeit
model = gp.Model("min_costs")

wind_costs = 1
solar_costs = 1
electrolizer_costs = 1
battery_cost = 0.0001

electrolizer_efficiency = 1

wind_size = model.addVar(name="wind_size")
solar_size = model.addVar(name="solar_size")
electrolyzer_size = model.addVar(name="electrolyzer_size")
battery_size = model.addVar(name="electrolyzer_size")

battery_rate_in = 1
battery_rate_out = 1
battery_efficiency = 0.8

battery_level = model.addVars(8760)
battery_out = model.addVars(8760)

electrolyzer_in = model.addVars(8760)

model.update()  # workaround: https://stackoverflow.com/a/66212311/859591

costs = (wind_costs * wind_size +
         solar_size * solar_costs +
         electrolyzer_size * electrolizer_costs +
         battery_size * battery_cost)

wind_profile = np.zeros(8760)  #np.random.random(8760)
wind_profile[::2] = 1
solar_profile = np.zeros(8760)  #np.random.random(8760)

power_generation_profile = wind_profile * wind_size + solar_profile * solar_size

for t in range(8760):
    model.addConstr(battery_level[t] <= battery_size)
    model.addConstr(electrolyzer_in[t] <= electrolyzer_size)
    model.addConstr(electrolyzer_in[t] <= power_generation_profile[t] + battery_efficiency * battery_out[t])
    
    model.addConstr(battery_out[t] <= battery_level[t])
    model.addConstr(battery_out[t] <= battery_rate_out)


model.addConstr(battery_level[0] == 0)
for t in range(1, 8760):
    model.addConstr(battery_level[t] <= power_generation_profile[t-1] + battery_level[t-1] - battery_out[t-1])
    model.addConstr(battery_level[t] <= battery_rate_in + battery_level[t-1])


hyrdogen_yearly = gp.quicksum(electrolyzer_in) * electrolizer_efficiency

    
model.addConstr(1e5 <= hyrdogen_yearly, "energy_production_minimum")
model.setObjective(costs, gp.GRB.MINIMIZE)

model.optimize()

5.38 s ± 428 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
battery_level

{0: <gurobi.Var C4 (value 0.0)>,
 1: <gurobi.Var C5 (value 1.0)>,
 2: <gurobi.Var C6 (value 1.0)>,
 3: <gurobi.Var C7 (value 2.0)>,
 4: <gurobi.Var C8 (value 1.0)>,
 5: <gurobi.Var C9 (value 2.0)>,
 6: <gurobi.Var C10 (value 1.0)>,
 7: <gurobi.Var C11 (value 2.0)>,
 8: <gurobi.Var C12 (value 1.0)>,
 9: <gurobi.Var C13 (value 2.0)>,
 10: <gurobi.Var C14 (value 1.0)>,
 11: <gurobi.Var C15 (value 2.0)>,
 12: <gurobi.Var C16 (value 1.0)>,
 13: <gurobi.Var C17 (value 2.0)>,
 14: <gurobi.Var C18 (value 1.0)>,
 15: <gurobi.Var C19 (value 2.0)>,
 16: <gurobi.Var C20 (value 1.0)>,
 17: <gurobi.Var C21 (value 2.0)>,
 18: <gurobi.Var C22 (value 1.0)>,
 19: <gurobi.Var C23 (value 2.0)>,
 20: <gurobi.Var C24 (value 1.0)>,
 21: <gurobi.Var C25 (value 2.0)>,
 22: <gurobi.Var C26 (value 1.0)>,
 23: <gurobi.Var C27 (value 2.0)>,
 24: <gurobi.Var C28 (value 1.0)>,
 25: <gurobi.Var C29 (value 2.0)>,
 26: <gurobi.Var C30 (value 1.0)>,
 27: <gurobi.Var C31 (value 2.0)>,
 28: <gurobi.Var C32 (value 1.0)>,
 

In [21]:
wind_size

<gurobi.Var wind_size (value 22.831050228310502)>

In [22]:
solar_size

<gurobi.Var solar_size (value 0.0)>

In [23]:
battery_size

<gurobi.Var electrolyzer_size (value 11.415525114155251)>

In [31]:
model.x

[22.831050228310502,
 0.0,
 11.415525114155251,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415525114155251,
 0.0,
 11.415

In [37]:
duration_one_run = 100e-3
samples = 1000
#samples = 100_000

to_days = 1 / 3600 / 24

cores = 20 * 32                                                            

duration_one_run * samples * to_days / cores * 5**6 

0.028257016782407406